In [125]:
import os
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import LinearSVC
from scipy.stats import uniform
from sklearn.metrics import confusion_matrix

In [126]:
filename='data/data_info.pkl'
with open(filename, 'rb') as handle:
    info=pickle.load(handle)
    
keys=info.keys()
GO_terms=[]
genes=[]
gene_image_file=[]
gene_image_nz=[]
images_list=[]
for key in keys:
    genes.append(info[key][0])
    gene_image_nz.append(info[key][1])
    gene_image_file.append(key)
    images_list.append(info[key][2])
    GO_terms.append(info[key][3])
    
#delete images/genes which are in the lowest 20% non-zero count
geneNb=len(info)
genes_array = np.array(genes)
image_files = np.array(gene_image_file)
image_nz = np.array(gene_image_nz)
k = int(geneNb/5)
idx = np.argpartition(image_nz, k)
id_smallest = idx[:k]

image_files_rem = image_files[idx[k:]]
image_nz_rem = image_nz[idx[k:]]
genes_rem = genes_array[idx[k:]]
GO_terms_rem=np.asarray(GO_terms)[idx[k:]]
images_list_rem=np.asarray(images_list)[idx[k:]]


In [127]:
from skimage.transform import resize
data_list=[]
for gray_img in images_list_rem:
    img=resize(gray_img, (140,300), anti_aliasing=True) #resize all images to 140x300
    img=img*255 #multiply by 255 to scale images back to the same integer value distribution
    img=np.asarray(img, dtype='uint8')
    data_list.append(img)


/home/desgab/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/comp551/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [128]:
data_array=np.asarray(data_list)
shape=data_array.shape
X=data_array.reshape(shape[0], shape[1]*shape[2])

In [129]:
X=X[:2000,:]

In [130]:
from sklearn.decomposition import PCA
pca=PCA()
pca.fit(X)
transformed=pca.transform(X)


PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [165]:
var_explained=pca.explained_variance_ratio_
sum_ve=0
num_comp=0
while(sum_ve<0.95):
    sum_ve+=var_explained[num_comp]
    num_comp+=1
print(num_comp)

303


In [166]:
X_pca=transformed[:,:num_comp]

In [167]:
filename='data/SIFT_data.pkl'
with open(filename, 'rb') as handle:
    SIFT_dict=pickle.load(handle)
GO_terms_rem=SIFT_dict['GO_terms']
GO_terms=GO_terms_rem[:2000]

In [168]:
'''
PICK THE GO TERMS THAT SHOW UP IN BETWEEN 50 and 500 Slices
'''

GO_annotations_nb = {}
for GO_list in GO_terms:
    for GO_term in GO_list:
        if GO_term in GO_annotations_nb:
            GO_annotations_nb[GO_term] += 1
        else:
            GO_annotations_nb[GO_term] = 1

list50to500GO = []
list50to500values = []
for GO in GO_annotations_nb.keys():
    if GO_annotations_nb[GO] > 51 and GO_annotations_nb[GO] < 501:
        list50to500GO.append(GO)
        list50to500values.append(GO_annotations_nb[GO])


In [169]:
GO_list=[]
for GO_term in list50to500GO:
    image_list=[]
    for image_GO in GO_terms:
        if GO_term in image_GO:
            image_list.append(1)
        else:
            image_list.append(0)
    GO_list.append(image_list)

GO_term_array=np.asarray(GO_list)

In [170]:
def train_valid_split(X,Y):
    idx_1=np.where(Y==1)[0]
    np.random.shuffle(idx_1)
    idx_0=np.where(Y==0)[0]
    np.random.shuffle(idx_0)
    train_idx=np.concatenate([idx_1[:int(len(idx_1)*0.8)],idx_0[:int(len(idx_0)*0.8)]])
    valid_idx=np.concatenate([idx_1[int(len(idx_1)*0.8):],idx_0[int(len(idx_0)*0.8):]])

    Y_train=Y[train_idx]
    Y_valid=Y[valid_idx]
    X_train=X[train_idx,:]
    X_valid=X[valid_idx,:]
    return X_train, X_valid, Y_train, Y_valid

In [175]:
from sklearn import metrics
Y=GO_term_array[1,:]
#X=genes_features
X=X_pca
X_train, X_valid, Y_train, Y_valid=train_valid_split(X,Y)

from sklearn.linear_model import LogisticRegression
#model=LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=2.0, random_state=0, max_iter=1000)
model=LinearSVC(loss='squared_hinge', penalty='l2',C=10, max_iter=3000, random_state=0)
model.fit(X_train,Y_train)
Y_pred=model.predict(X_valid)
score = f1_score(Y_valid, Y_pred)
print(score)
score = metrics.roc_auc_score(Y_valid, Y_pred)
print(score)
Y_pred=model.predict(X_train)
score = f1_score(Y_train, Y_pred)
print(score)
score = metrics.roc_auc_score(Y_train, Y_pred)
print(score)


0.09523809523809523
0.5252824469550841
0.33333333333333337
0.6987148002619515


/home/desgab/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/comp551/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [163]:
model=LinearSVC(loss='squared_hinge', penalty='l2',C=3, max_iter=3000, random_state=0)

X=X_pca
scores=[]
for i in range(GO_term_array.shape[0]):
    Y=GO_term_array[i,:]
    X_train, X_valid, Y_train, Y_valid=train_valid_split(X,Y)
    model.fit(X_train,Y_train)
    Y_pred=model.predict(X_valid)
    score = metrics.roc_auc_score(Y_valid, Y_pred)
    scores.append(score)
    print(score)
    

/home/desgab/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/comp551/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5444100447196422


/home/desgab/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/comp551/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5707495177734913


/home/desgab/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/comp551/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


...

/home/desgab/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/comp551/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.4252577319587629


/home/desgab/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/comp551/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5136482939632545


KeyboardInterrupt: 

In [59]:
svm=LinearSVC(loss='squared_hinge', penalty='l2',C=10, max_iter=3000, random_state=0)
svm.fit(X_train,Y_train)
Y_pred=svm.predict(X_valid)

from sklearn.metrics import f1_score
score = f1_score(Y_valid, Y_pred)
print(score)

0.015037593984962405


/home/desgab/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/comp551/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [46]:
pipe_LinearSVC = Pipeline([
#    ('norm', Normalizer(norm='l2')), #normalize the features with a l2 norm
    ('clf', LinearSVC(loss='squared_hinge', penalty='l2',C=1.83, random_state=0)), #C is the inverse of regularization strength
])

In [47]:
'''
Searching for hyperparameters with Randomized search
'''

params = {"clf__C": uniform(0.5, 1.5),
          "clf__loss": ['squared_hinge', 'hinge']}

seed = 551 # Setting a constant seed for repeatability. min_df
num_iter=2
cv=2 #number of cross-validation folds
pipe=pipe_LinearSVC 

random_search = RandomizedSearchCV(pipe, param_distributions = params, scoring='f1',cv=cv, verbose = 10, random_state = seed, n_iter = num_iter)
random_search.fit(X, Y)

#getting cross validation results
results=random_search.cv_results_
data = {"clf__C": results.get('param_clf__C').data,
          "clf__loss": results.get('param_clf__loss').data,
          "mean_test_score": results.get('mean_test_score').data,}



Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] clf__C=1.4600960718420557, clf__loss=hinge ......................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/desgab/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/comp551/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]  clf__C=1.4600960718420557, clf__loss=hinge, score=0.036231884057971016, total=   0.8s
[CV] clf__C=1.4600960718420557, clf__loss=hinge ......................


/home/desgab/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/comp551/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.6s remaining:    0.0s


[CV]  clf__C=1.4600960718420557, clf__loss=hinge, score=0.04363636363636364, total=   0.7s
[CV] clf__C=1.4600190730326272, clf__loss=squared_hinge ..............


/home/desgab/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/comp551/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.7s remaining:    0.0s


[CV]  clf__C=1.4600190730326272, clf__loss=squared_hinge, score=0.03322259136212625, total=   1.0s
[CV] clf__C=1.4600190730326272, clf__loss=squared_hinge ..............


/home/desgab/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/comp551/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    3.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    3.6s finished


[CV]  clf__C=1.4600190730326272, clf__loss=squared_hinge, score=0.04285714285714286, total=   0.9s


/home/desgab/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/comp551/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


***MAKING 5-FOLD CROSS-VALIDATION

In [ ]:
from sklearn.model_selection import cross_val_score
LR_scores=cross_val_score(pipe_logistic_regression, all_X_train, all_Y_train, scoring='accuracy', cv=5)
print(LR_scores)